# Database

In [1]:
# Imports
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from inferpy.data import mnist
import numpy as np
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture
# seed
np.random.seed(2020)

Using TensorFlow backend.


Load and prepare Mnist database form InferPy's package.

In [2]:
# Load data from InferPy
(X, y), _ = mnist.load_data(num_instances=1000, digits=[1,4,7])
# Create dataframe
dataset = pd.DataFrame(data=X)
dataset["number"] = y
# Split dataframe
X = dataset.drop(['number'], axis=1)
y = dataset['number']

# Constants

In [3]:
# Ammount of samples
n_samples = X.shape[0]
# Observed space dimensionality
data_dim = X.shape[1]
# Ammount of classes
n_classes = len(y.unique())

# Learning

Create `BayesianGaussianMixture` object from [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html), with a number of components equal to the ammount of classes.

The parameter `weight_concentration_prior_type` models if a Dirichlet distribution or a Dirichlet Process is used for the weights. In this case, we model using a distribution. 

The following prior values are used:
- The means variable $\mu$ follows a centered normal distribution: 
  $$\mu_k \sim \mathcal{N}_{data\_dim}(0,I).$$
- The precision variables $\Lambda$ follow a Wishart distribution with parameters
  $$\Lambda_k \sim \mathcal{W}_{data\_dim}(data\_dim, I).$$
- The weights concentration variable follows a dirichlet with parameter
  $$ \pi \sim \text{Symmetric-Dirichlet}\Big(\frac{1}{n\_classes}\Big).$$
    

In [4]:
gm = BayesianGaussianMixture(n_components = n_classes, 
                             weight_concentration_prior_type = 'dirichlet_distribution',
                             weight_concentration_prior = 1/n_classes,
                             mean_precision_prior = 1,
                             mean_prior = np.zeros([data_dim]),
                             degrees_of_freedom_prior = data_dim,
                             covariance_prior = np.identity(data_dim),
                             max_iter = 1000,
                             tol = 1e-3,
                             random_state = 0)

We fit the model. The stop criteria is either reaching `max_iter` steps on the EM algorithm or having a lower bound difference betweeen iterations below `tol`.

In [5]:
gm.fit(X)

BayesianGaussianMixture(covariance_prior=array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]]),
                        degrees_of_freedom_prior=784, max_iter=1000,
                        mean_precision_prior=1,
                        mean_prior=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0....
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]),
                        n_components=3, random_state=0,
         

The most probable label for each datapoint is calculated using `predict`, we might calculate the acurated estimations as:

In [6]:
print(sum(gm.predict(X) == y)/n_samples)

0.339


The model's posterior parameters might be inspected via `weights_`, `means_` and `precisions_`

In [7]:
print(gm.weights_)

[0.26806527 0.38994339 0.34199134]


In [8]:
print(gm.means_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
print(gm.precisions_)

[[[1051.71813954    0.            0.         ...    0.
      0.            0.        ]
  [   0.         1051.71813954    0.         ...    0.
      0.            0.        ]
  [   0.            0.         1051.71813954 ...    0.
      0.            0.        ]
  ...
  [   0.            0.            0.         ... 1051.71813954
      0.            0.        ]
  [   0.            0.            0.         ...    0.
   1051.71813954    0.        ]
  [   0.            0.            0.         ...    0.
      0.         1051.71813954]]

 [[1173.5423185     0.            0.         ...    0.
      0.            0.        ]
  [   0.         1173.5423185     0.         ...    0.
      0.            0.        ]
  [   0.            0.         1173.5423185  ...    0.
      0.            0.        ]
  ...
  [   0.            0.            0.         ... 1173.5423185
      0.            0.        ]
  [   0.            0.            0.         ...    0.
   1173.5423185     0.        ]
  [   0.      